In [36]:
import pandas as pd
import glob
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import lightgbm as lgbm
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
import datetime
import warnings
warnings.simplefilter("ignore")

In [52]:
def get_articles(name):
    contents = []
    list_of_files = glob.glob('../data/news/'+name+'/*.txt')        
    for i,file_name in enumerate(list_of_files):
        id_article = file_name[:-4].split('/')[4]
        contents.append(open(file_name,'r').read())
    return contents

def get_Y_train():
    true_val = []
    indexes = []
    articles = open('../data/labels_training.txt','r').readlines()[1:]
    for i in articles:
        true_val.append(int(i.split(',')[1][:-1]))
        indexes.append(i.split(',')[0])
    return pd.DataFrame(true_val,columns=['fake_news'],index=indexes)

### Get articles for training and testing

In [53]:
train = get_articles('training')
test = get_articles('test')

In [54]:
def sentense2cleanTokens(sent):
    sent = sent.lower()
    sent = "".join([x if x.isalpha() else " " for x in sent])
    sent = " ".join(sent.split())
    return sent

X_train = []
for i in train:
    X_train.append(sentense2cleanTokens(i))
    
X_test = []
for j in test:
    X_test.append(sentense2cleanTokens(j))
    
Y_train = get_Y_train().sort_index()

In [57]:
tfidf_v = TfidfVectorizer(max_df=0.95, min_df=5, max_features=1000, stop_words='english')
train_tf_idf = tfidf_v.fit_transform(X_train)
test_tf_idf = tfidf_v.fit_transform(X_test)

In [58]:
model = GradientBoostingClassifier()
pred = np.mean(cross_val_score(model, train_tf_idf, Y_train, cv=10))
print(pred)

0.4935087719298245


### It's prediction time !

In [45]:
clf = GradientBoostingClassifier()
clf.fit(train_tf_idf,Y_train)
Y_test = clf.predict(test_tf_idf)

### Create a submission file

In [ ]:
now = str(datetime.datetime.now())[:19]
with open('../data/submissions/bogota_mates_submission_'+now+'.txt', 'w') as f:
    f.write('doc,class\n')
    for value,doc in zip(Y_test,X_test.index):
        f.write(str(doc)+ ',' + str(value)+"\n")